In [2]:
api_key = "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"

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=1,
    groq_api_key = api_key
)

response = llm.invoke("Generate a format for cold email to H.R for a specific job profile")

In [25]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-34832?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Data Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsLead Data EngineerKarnataka, IndiaBecome a Part of the NIKE, Inc. Team
NIKE, Inc. does more 

In [26]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED INFORMATION FROM WEBSITE
    {page_data}
    ### INSTRUCTION
    this is extracted text from a career page of website.
    Your job is to extract extract the job posting and return them in json format containing  following keys
    : 'role' , 'skills' , 'description'.
    Only return the valid json 
    ### Valid Json (NO PREAMBLE)
    """
)

chain_extract = prompt_extract|llm
res = chain_extract.invoke(input={'page_data':page_data})

In [27]:
print(res.content)

{"role": "Lead Data Engineer", "skills": ["AWS web Services", "Databricks", "EMR", "Python", "Microservices", "CI/CD pipeline", "Jenkins", "Terraform", "Docker", "Kubernetes", "OOP Concepts", "Data Structures", "Design Patterns", "Web Services", "Caching", "Scalability", "REST Principles", "Databases", "Computer science fundamentals", "Data Governance"], "description": "We are looking for a Lead Data Platform Engineer who excels in team environments and are excited about building cloud native platforms that can scale with the demand of our business. This role is part of Enterprise Architecture and Developer Experience aggressively innovates solutions to drive growth while creating and implementing tools that help make everything else in the company possible."}


In [13]:
print(type(res.content))

<class 'str'>


In [28]:
from langchain_core.output_parsers  import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Data Engineer',
 'skills': ['AWS web Services',
  'Databricks',
  'EMR',
  'Python',
  'Microservices',
  'CI/CD pipeline',
  'Jenkins',
  'Terraform',
  'Docker',
  'Kubernetes',
  'OOP Concepts',
  'Data Structures',
  'Design Patterns',
  'Web Services',
  'Caching',
  'Scalability',
  'REST Principles',
  'Databases',
  'Computer science fundamentals',
  'Data Governance'],
 'description': 'We are looking for a Lead Data Platform Engineer who excels in team environments and are excited about building cloud native platforms that can scale with the demand of our business. This role is part of Enterprise Architecture and Developer Experience aggressively innovates solutions to drive growth while creating and implementing tools that help make everything else in the company possible.'}

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('my_portfolio.csv')

In [10]:
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [16]:
import chromadb

client = chromadb.PersistentClient('vetor_database')
collection = client.create_collection(name = "portfolio")

In [17]:
for index, row in df.iterrows():
    tech_stack = row['Techstack'] 
    link = row['Links']  
    
    # Store metadata 
    collection.add(
        documents = tech_stack, 
        metadatas=[{ "links": link}],
        ids=[f"id_{index}"]
    )

print("All records (tech stack and links) stored in Chroma DB.")

All records (tech stack and links) stored in Chroma DB.


In [21]:
results = collection.query(
    query_texts= "django mysql",
    n_results=1,
    
)
results

{'ids': [['id_3']],
 'distances': [[0.17932198553737616]],
 'metadatas': [[{'links': 'https://example.com/python-portfolio'}]],
 'embeddings': None,
 'documents': [['Python, Django, MySQL']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [29]:
json_res['skills']

['AWS web Services',
 'Databricks',
 'EMR',
 'Python',
 'Microservices',
 'CI/CD pipeline',
 'Jenkins',
 'Terraform',
 'Docker',
 'Kubernetes',
 'OOP Concepts',
 'Data Structures',
 'Design Patterns',
 'Web Services',
 'Caching',
 'Scalability',
 'REST Principles',
 'Databases',
 'Computer science fundamentals',
 'Data Governance']

In [33]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION :
    {job_description}
    ### INSTRUCTION:
    You are a employ of service based startup named spy .Spy  is an AI & Software Consulting company dedicated to facilitating
    he seamless integration of business processes through automated tools. 
    Your job is to write a cold email for the job role which is given in job description.Also add relevant portfolio links in the email:{link_list}
    Do Not Provide Preamble.
    ### Email(no preamble)
    """
)

In [36]:
results = collection.get()
links = [metadata['links'] for metadata in results['metadatas']]
print(links)

['https://example.com/react-portfolio', 'https://example.com/angular-portfolio', 'https://example.com/android-portfolio', 'https://example.com/kotlin-android-portfolio', 'https://example.com/android-tv-portfolio', 'https://example.com/ios-ar-portfolio', 'https://example.com/xamarin-portfolio', 'https://example.com/kotlin-backend-portfolio', 'https://example.com/typescript-frontend-portfolio', 'https://example.com/full-stack-js-portfolio', 'https://example.com/ml-python-portfolio', 'https://example.com/devops-portfolio', 'https://example.com/vue-portfolio', 'https://example.com/python-portfolio', 'https://example.com/java-portfolio', 'https://example.com/flutter-portfolio', 'https://example.com/wordpress-portfolio', 'https://example.com/magento-portfolio', 'https://example.com/react-native-portfolio', 'https://example.com/ios-portfolio']


In [37]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description" : str(json_res),"link_list":links})


In [39]:
print(res.content)

Subject: Lead Data Engineer Application for Cloud-Native Platform Development

Dear Hiring Manager,

I came across the Lead Data Engineer position at your esteemed organization and was impressed by the company's commitment to innovation and growth. As a seasoned data engineer with a passion for building scalable cloud-native platforms, I'm excited to apply for this role.

With expertise in AWS web services, Databricks, EMR, and containerization using Docker, I've successfully designed and implemented data pipelines that have resulted in significant increases in data processing efficiency. My proficiency in CI/CD pipelines using Jenkins has enabled teams to automate testing and deployment, reducing time-to-market and improving overall product quality.

As a strong believer in the importance of data governance, I've implemented scalable data architectures that ensure data integrity, security, and compliance. My experience with Terraform has allowed me to manage infrastructure as code, st